In [1]:
#########################################################################
#Data Collection 
#
# Functions for handling subwatershed characteristics data collection
#
# Authors: Shannon McAvoy (smcavoy@dewberry.com)
#
# Editor: Jason Matney (jmatney@dewberry.com)
#
# Copyright: Dewberry Engineers Inc.
#########################################################################

In [2]:
import pandas as pd
import arcpy
import numpy as np
from numpy import mean
from numpy import std
import os
from arcpy import management
from arcgis.gis import GIS
from time import time

In [3]:
# Login for ArcPro, AGOL or API Server 
#gis = GIS("https://www.arcgis.com", username="dpiazza_dewberry", password="*****")

In [4]:
path = "C:\\Users\jmatney\Documents\GitHub\IndianaRisk\data\Indiana_ML"
hucs = "C:\\Users\jmatney\Documents\GitHub\IndianaRisk\data\shp"

In [5]:
arcpy.env.overwriteOutput = True

In [6]:
st = time()

# Set Local Variables
output_workspace = os.path.join(path, "working_dsn")

arcpy.env.workspace = output_workspace

##### change these based on what subwatersheds you want #####

# all_subwatersheds = r"P:\Temp\McAvoy\ML_DataCollection\Indiana_ML\Indiana_HUC12_groups\1.shp"
all_subwatersheds = os.path.join(hucs, "wbdhu12_a_IN_only.shp")

# All centroids of subwatersheds
all_centroids = os.path.join(hucs, "wbdhu12_a_IN_only_centroids.shp")

#set source for streams data
streams = os.path.join(path, "Indiana_50c_streams_edited\Hydrography_HighRes_FlowLine_NHD_USGS.shp")

#####these are the same for all subwatersheds in Indiana#####

#set source for dem
dem_Indiana = os.path.join(path, "IN_EXTENT_MOSAIC\IN_EXTENT_MOSAIC.tif")

#set source for slope dem
dem_slope = os.path.join(path, "Indiana_SlopeRaster\IN_ext_slope")

#set source for NHFL Data
nfhl_sfha = os.path.join(path, "NFHL_18_20200310.gdb\S_FLD_HAZ_AR")


#set source for water bodies data 
water_bodies = os.path.join(path, "IndianaMAP_WaterBodies\Water_Bodies_Lakes_LocalRes\Hydrography_LocalRes_WaterbodyDiscrete_NHD_IN.shp")

#set source for dams data 
dams = os.path.join(path, "IndianaMAP_Dams\Dams_IDNR\Dams_IDNR_IN.shp")

#set source for bridges data
bridges = os.path.join(path, "IndianaMAP_Bridges\Bridges_County_INDOT\Bridges_County_INDOT_IN.shp")

#set source for streets data
streets = os.path.join(path, "IndianaMAP_Streets\Streets_Centerlines_IGIO\County_Street_Centerlines_IGIO_IN.gdb\County_Street_Centerlines_IGIO_IN_Dec2019")


#set source for railraods data
railroads = os.path.join(path, "IndianaMAP_Railroads\Railroads_Active_Abandoned_INDOT\Rail_System_Active_Abandoned_INDOT_IN.shp")

#set source for ACS population data
population = os.path.join(path, "Indiana_PopulationData\Indiana_PopulationData.shp")

#set source for ACS median income data 
median_income = os.path.join(path, "Indiana_IncomeData\Indiana_IncomeData.shp")

#set source for county boundary data
county_boundary = os.path.join(path, "MarionCounty_Boundary\Marion_County_Boundary.shp")

#source for Bing building footprints for Indiana (attrubuted with open street maps data)
building_footprints = os.path.join(path, "BuildingFootprints_Indiana\Building_Footprints_Attributed_IN.shp")

#folder with partial duration files, set as workspace temporarily, then reset when done
directory_rainfall = os.path.join(path, "MarionCounty_Rainfall\All_Rainfall_Clipped_IN")

#set source for nlcd land use data
lu_usa = os.path.join(path, r"NLCD_Impervious\NLCD_indiana_polygon.shp")

#set source for impervious indicator data
impervious_usa = os.path.join(path, "NLCD_Impervious\\NLCD_2016_Impervious_L48_20190405_PERCENT\\NLCD_2016_Impervious_L48_20190405.img")



subwatershed_list = []
area_list = []
perimeter_list = []
watershed_length_list = []
elongation_ratio_list = []
shape_factor_list = []
circulatory_ratio_list = []
relief_list = []
relief_ratio_list = []
avg_slope_list = []
drainage_density_list = []
ruggedness_list = []
aae_list = []
buildings_aae_list = []
x_list = []
buildings_x_list = []
water_bodies_list = []
dams_list = []
bridges_list = []
streets_list = []
railroads_list = []
population_list = []
dependent_population_list = []
population_density_list = []
avg_median_income_list = []
housing_density_list = []
population_change_list = []
dist_to_stream_avg_list = []
dist_to_stream_stdev_list = []

lu_21_list = []
lu_22_list = []
lu_23_list = []
lu_24_list = []
lu_41_list = []
lu_82_list = []
impervious_percent_list = []

orb100yr06h_list = []
orb100yr12h_list = []
orb100yr24h_list = []
orb25yr06h_list = []
orb25yr12h_list = []
orb25yr24h_list = []
orb2yr06h_list = []
orb2yr12h_list = []
orb2yr24h_list = []
orb50yr06h_list = []
orb50yr12h_list = []
orb50yr24h_list = []
orb100yr06ha_am_list = []
orb100yr12ha_am_list = []
orb100yr24ha_am_list = []
orb25yr06ha_am_list = []
orb25yr12ha_am_list = []
orb25yr24ha_am_list = []
orb2yr06ha_am_list = []
orb2yr12ha_am_list = []
orb2yr24ha_am_list = []
orb50yr06ha_am_list = []
orb50yr12ha_am_list = []
orb50yr24ha_am_list = []


#########################################################################################################
print(round(((time()-st)/60), 2) , 'minutes to process.')

0.0 minutes to process.


In [7]:
# GLOBAL VARIABLES
LOG_NAME = 'DataCollection_export.log'

In [8]:
# #search cursor through each row of county subwatersheds file
# rows = arcpy.SearchCursor(all_subwatersheds)

# row_count = 0
# for row in rows:
    
#     row_count += 1
    
#     subwatershed_number = row.getValue("HUC12")
    
#     subwatershed_list.append(subwatershed_number)
    
#     print("------ Subwatershed:", subwatershed_number, "Number", row_count, "of 1590 ------")              
    

In [9]:
completed_hucs = pd.read_excel(r"C:\Users\jmatney\Documents\GitHub\IndianaRisk\data\output\completed_hucs.xlsx", converters={'subwatershed': lambda x: str(x)})

In [10]:
done = completed_hucs['subwatershed'].str.strip('[]').tolist()

In [11]:
len(done)

532

In [12]:
st = time()

row_count=len(done)
with arcpy.da.SearchCursor(all_subwatersheds, ['HUC12']) as cursor:

    for row in cursor:
        if row[0] not in done:
            subwatershed_number = row[0]

            row_count += 1

            subwatershed_list.append(subwatershed_number)

            print("------ Subwatershed:", subwatershed_number, "Number", row_count, "of 1590 ------")              

            #select subwatershed from shapefile with all of them
            subwatershed_selection = arcpy.SelectLayerByAttribute_management(all_subwatersheds, "NEW_SELECTION",
                                                                             "HUC12 = " + "'"+subwatershed_number+"'")

            #select centroid from shapefile with all of them
            centroid_selection = arcpy.SelectLayerByAttribute_management(all_centroids, "NEW_SELECTION",
                                                                             "HUC12 = " + "'"+subwatershed_number+"'")
            #copy selected subwatershed to it's own file
            subwatershed = arcpy.CopyFeatures_management(subwatershed_selection, "subwatershed")

            #copy selected centroid to it's own file
            centroid = arcpy.CopyFeatures_management(centroid_selection, "centroid")

            #clip dem to subwatershed area
            # dem_clip = arcpy.Clip_management(dem_Indiana, "#", "dem_clip.tif", subwatershed, "#" , "ClippingGeometry", "NO_MAINTAIN_EXTENT")
            dem_clip =  os.path.join(path, "working_dsn\dem_clip.tif")
            arcpy.Clip_management(dem_Indiana, "#", dem_clip, subwatershed_selection)
            print("clipped dem to subwatershed area")

            #calculate area of subwatershed
            arcpy.AddField_management(subwatershed, "AREA", "DOUBLE")

            area = arcpy.CalculateGeometryAttributes_management(subwatershed, "AREA AREA_GEODESIC", '', 
                                                                "SQUARE_KILOMETERS",
                                                                None)

            #print area
            rows = arcpy.SearchCursor(area)

            for row in rows:
                area = row.getValue("AREA")

            area_list.append(area)
            print(area, " square kilometers")


            #calculate perimeter of subwatershed
            arcpy.AddField_management(subwatershed, "PERIMETER", "DOUBLE")
            perimeter = arcpy.CalculateGeometryAttributes_management(subwatershed, "PERIMETER PERIMETER_LENGTH_GEODESIC",
                                                                     "KILOMETERS")

            print("perimeter calculated")

            #print perimeter
            rows = arcpy.SearchCursor(perimeter)

            for row in rows:
                perimeter = row.getValue("PERIMETER")

            perimeter_list.append(perimeter)
            print(perimeter, " kilometers")



            #get average slope using zonal statistics from slope dem
            avg_slope = arcpy.sa.ZonalStatistics(subwatershed, "FID", dem_slope, "MEAN")

            #get slope value
            avg_slope_result = arcpy.GetRasterProperties_management(avg_slope, "MAXIMUM")

            avg_slope_value = avg_slope_result.getOutput(0)

            avg_slope_list.append(avg_slope_value)

            print(avg_slope_value, " = avg slope (%)")


            #calculate circulatory ratio
            #ratio of area to the area of a circle having equal perimeter as the perimeter of drainage basin

            #area of a circle with same perimeter as above
            #C = 2(pi)r
            #r = C/(2pi)
            #A = (pi)r^2
            circle_radius = perimeter/(2*numpy.pi)
            print(circle_radius, "is the radius of a circle with the same perimeter.")
            circle_area = (numpy.pi*(circle_radius**2))
            print(circle_area, "is the area of a circle with the same perimeter.")

            #ratio of subwatershed perimeter to circle circumference
            circulatory_ratio = area / circle_area

            circulatory_ratio_list.append(circulatory_ratio)
            print(circulatory_ratio, " is the circulatory ratio")


            #calculate relief
            #elevation difference before basin outlet and highest point located in the perimeter of basin

            #find highest point on perimeter

            #use Raster Domain tool to get z-enabled polyline of perimeter
            perimeter_polyline = arcpy.RasterDomain_3d(dem_clip, "perimeter_polyline.shp", "LINE")
            print("created 3d polyline of subwatershed perimeter")

            #Convert each vertices of the polyline into points
            perimeter_points = arcpy.FeatureVerticesToPoints_management(perimeter_polyline, "perimeter_points.shp")
            print("created perimeter points")

            #get Z values into the attribute table for the points
            perimeter_points_Z = arcpy.AddZInformation_3d(perimeter_points, "Z")
            print("added z information")

            #get the max value from all of the points
            perimeter_stats = arcpy.Statistics_analysis(perimeter_points_Z, "perimeter_stats", [["Z", "MAX"], ["Z", "MIN"]])

            # Get a list of field names to display
            field_names = [i.name for i in arcpy.ListFields(perimeter_stats) if i.type != 'OID']

            # Open a cursor to extract results from stats table
            cursor = arcpy.da.SearchCursor(perimeter_stats, field_names)

            # Create a pandas dataframe to display results
            df = pd.DataFrame(data=[row for row in cursor],
                                  columns=field_names)

            print(df)

            #get the values for max Z and min Z into a format to use them

            rows = arcpy.SearchCursor(perimeter_stats)

            for row in rows:
                max_z = row.getValue("MAX_Z")
                min_z = row.getValue("MIN_Z")


            print(max_z)
            print(min_z)

            relief = max_z - min_z
            print("The relief of the subwatershed is: ", relief," meters.")

            relief_list.append(relief)



            #find area covered by A and AE zones and X NFHL Zones

            #clip to subwatershed area
            nfhl_sfha_clip = arcpy.Clip_analysis(nfhl_sfha, subwatershed, "nfhl_sfha.shp")

            #select Zone A and Zone AE
            nfhl_sfha_aae_selection = arcpy.management.SelectLayerByAttribute(nfhl_sfha_clip, "NEW_SELECTION", 
                                                               "FLD_ZONE = 'AE' Or FLD_ZONE = 'A'", None)

            #copy features to new feature class
            nfhl_sfha_aae = arcpy.CopyFeatures_management(nfhl_sfha_aae_selection, "nfhl_sfha_aae")

            #add new field for area
            nfhl_sfha_aae_addfield = arcpy.AddField_management(nfhl_sfha_aae, "AREA", "DOUBLE")

            #calculate the area for each of the features
            aae_area = arcpy.CalculateGeometryAttributes_management(nfhl_sfha_aae_addfield, "AREA AREA_GEODESIC", '', 
                                                                "SQUARE_KILOMETERS",
                                                                None)
            #get the sum of all of the areas
            aae_area_list = []  

            rows = arcpy.SearchCursor(aae_area)  
            for row in rows:  
                aae_area_feature = row.getValue("AREA")  
                aae_area_list.append(aae_area_feature)  

            aae_area_sum = sum(aae_area_list)

            aae_list.append(aae_area_sum)

            print(aae_area_sum, " = area of all AE and A zones in the subwatershed (square km)")



            #select Zone X, 0.2% chance flood area
            nfhl_sfha_x_selection = arcpy.management.SelectLayerByAttribute(nfhl_sfha_clip, "NEW_SELECTION", 
                                                               "FLD_ZONE = 'X' And ZONE_SUBTY = '0.2 PCT ANNUAL CHANCE FLOOD HAZARD'",
                                                                              None)

            #copy features to new feature class
            nfhl_sfha_x = arcpy.CopyFeatures_management(nfhl_sfha_x_selection, "nfhl_sfha_x")

            #add new field for area
            nfhl_sfha_x_addfield = arcpy.AddField_management(nfhl_sfha_x, "AREA", "DOUBLE")

            #calculate the area for each of the features
            x_area = arcpy.CalculateGeometryAttributes_management(nfhl_sfha_x_addfield, "AREA AREA_GEODESIC", '', 
                                                                "SQUARE_KILOMETERS",
                                                                None)
            #get the sum of all of the areas
            x_area_list = []  

            rows = arcpy.SearchCursor(x_area)  
            for row in rows:  
                x_area_feature = row.getValue("AREA")  
                x_area_list.append(x_area_feature)  

            x_area_sum = sum(x_area_list)

            x_list.append(x_area_sum)

            print(x_area_sum, " = area of all X zones, 0.2PCT zone subtype in the subwatershed (square km)")


            #calculate area covered by lakes/reserviors 

            #clip to subwatershed area
            water_bodies_clip = arcpy.Clip_analysis(water_bodies, subwatershed, "water_bodies.shp")

            #copy features to new feature class
            water_bodies_copy = arcpy.CopyFeatures_management(water_bodies_clip, "water_bodies_copy.shp")

            #add new field for area
            water_bodies_addfield = arcpy.AddField_management(water_bodies_copy, "AREA", "DOUBLE")

            #calculate the area for each of the features
            water_bodies_area = arcpy.CalculateGeometryAttributes_management(water_bodies_addfield, "AREA AREA_GEODESIC", '', 
                                                                "SQUARE_KILOMETERS",
                                                                None)
            #get the sum of all of the areas
            water_bodies_area_list = []  

            rows = arcpy.SearchCursor(water_bodies_area)  
            for row in rows:  
                wb_area_values = row.getValue("AREA")  
                water_bodies_area_list.append(wb_area_values)  

            water_bodies_area_sum = sum(water_bodies_area_list)

            water_bodies_list.append(water_bodies_area_sum)
            print(water_bodies_area_sum, " = area of all water bodies in the subwatershed (square km)")


            #count all of the dams in the subwatershed

            #clip dams to subwatershed
            dams_clip = arcpy.Clip_analysis(dams, subwatershed, "dams.shp")

            #count number of dam points
            dams_count = arcpy.GetCount_management(dams_clip)

            dams_list.append(dams_count)

            print(dams_count, " = number of dams in the subwatershed")



            #count all of the bridges in the subwatershed

            #clip bridges to subwatershed
            bridges_clip = arcpy.Clip_analysis(bridges, subwatershed, "bridges.shp")

            #count number of dam points
            bridges_count = arcpy.GetCount_management(bridges_clip)

            bridges_list.append(bridges_count)

            print(bridges_count, " = number of bridges in the subwatershed")



            #calculate the kilometers of streets in the subwatershed

            #clip streets to subwatershed area 
            streets_clip = arcpy.Clip_analysis(streets, subwatershed, "streets.shp")

            #add field to calculate length of each street
            streets_addfield = arcpy.AddField_management(streets_clip, "LENGTH_KM", "DOUBLE")

            #calculate the area for each of the features
            streets_length = arcpy.CalculateGeometryAttributes_management(streets_addfield, "LENGTH_KM LENGTH_GEODESIC", 
                                                                          'KILOMETERS')
            #get the sum of all of the areas
            streets_lengths_list = []  

            rows = arcpy.SearchCursor(streets_length)  
            for row in rows:  
                streets_lengths_values = row.getValue("LENGTH_KM")  
                streets_lengths_list.append(streets_lengths_values)  

            streets_length_sum = sum(streets_lengths_list)

            streets_list.append(streets_length_sum)

            print(streets_length_sum, " = sum of all streets in the subwatershed (km)")


            #calculate km of railroads in subwatershed

            #clip railroads to subwatershed area 
            railroads_clip = arcpy.Clip_analysis(railroads, subwatershed, "railroads.shp")

            #add field to calculate length of each railroad
            railroads_addfield = arcpy.AddField_management(railroads_clip, "LENGTH_KM", "DOUBLE")

            #calculate the area for each of the features
            railroads_length = arcpy.CalculateGeometryAttributes_management(railroads_addfield, "LENGTH_KM LENGTH_GEODESIC", 'KILOMETERS', 
                                                                "",
                                                                None)
            #get the sum of all of the areas
            railroads_lengths_list = []  

            rows = arcpy.SearchCursor(railroads_length)  
            for row in rows:  
                railroads_lengths_values = row.getValue("LENGTH_KM")  
                railroads_lengths_list.append(railroads_lengths_values)  

            railroads_length_sum = sum(railroads_lengths_list)

            railroads_list.append(railroads_length_sum)

            print(railroads_length_sum, " = sum of all railroads in the subwatershed (km)")   


            #ACS population data - 5 year estimates (2014-2018) gotten from ESRI Living Atlas Data
            #data is chosen to be on census tract level


            #clip to subwatershed
            population_clip = arcpy.Clip_analysis(population, subwatershed, "population.shp")

            #get total population
            #field = B01001_001E = Total Population (alias)
            #when clipped, field name changes to B01001_001

            #get the sum of all of the populations in each tract
            total_pop_list = []  

            rows = arcpy.SearchCursor(population_clip)  
            for row in rows:  
                total_pop_value = row.getValue('B01001_001') 
                total_pop_list.append(total_pop_value)  

            total_pop_sum = sum(total_pop_list)

            population_list.append(total_pop_sum)
            print(total_pop_sum, " = total population in the subwatershed") 

            #get the average of percentages of dependent age groups in each tract
            #field = B01001_calc_pctDependE = Percent of Population in Dependent Age Groups (under 18 and 65+) (alias)
            #when clipped field changes to B01001_61
            dependent_pop_list = []  

            rows = arcpy.SearchCursor(population_clip)  
            for row in rows:  
                dependent_pop_value = row.getValue('B01001__61') 
                dependent_pop_list.append(dependent_pop_value)  

            dependent_pop_avg_pct = mean(dependent_pop_list)

            dependent_population_list.append(dependent_pop_avg_pct)

            print(dependent_pop_avg_pct, " = total percent of dependent population in the subwatershed") 

            #find population density
            population_density = total_pop_sum / area

            population_density_list.append(population_density)

            print(population_density, " = population density of subwatershed (people/square km)")


            #ACS population data - 5 year estimates (2014-2018) gotten from ESRI Living Atlas Data
            #data is chosen to be on census tract level

            #clip to subwatershed
            median_income_clip = arcpy.Clip_analysis(median_income, subwatershed, "median_income.shp")

            #get average median income 
            #field = B19049_001 = Median Household Income in past 12 months 
            #^(inflation-adjusted dollars to last year of 5-year range) (alias)

            #get the sum of all of the populations in each tract
            total_median_income_list = []  

            rows = arcpy.SearchCursor(median_income_clip)  
            for row in rows:  
                median_income_value = row.getValue('B19049_001') 
                total_median_income_list.append(median_income_value)  

            median_income_average = mean(total_median_income_list)

            avg_median_income_list.append(median_income_average)

            print(median_income_average, " = average median income in the subwatershed") 


            #get housing density

            #clip building footprints to subwatershed area
            building_footprints_clip = arcpy.Clip_analysis(building_footprints, subwatershed, "building_footprints.shp")

            #select buildings that are marked residential
            building_footprints_residential = arcpy.SelectLayerByAttribute_management(building_footprints_clip, "NEW_SELECTION",
                                                                                 "RES_NONRES = 'Res'")

            #get count of how many buildings there are 
            buildings_count = arcpy.GetCount_management(building_footprints_residential)

            buildings_count_number = buildings_count.getOutput(0)
            print(buildings_count_number, " = number of residential building footprints in the subwatershed")

            #divide number of buildings by subwatershed area
            housing_density = int(buildings_count_number) / area

            housing_density_list.append(housing_density)

            print(housing_density, " = housing density (buildings per square km)")


           #find total population from the 2013 5-year ACS estimates
            #get the sum of all of the 2013 populations in each tract
            #field = DP05_0001E = TotalPopulation from the 2013 ACS 5yr estimates
            total_pop_list_2013 = []  

            rows = arcpy.SearchCursor(population_clip)  
            for row in rows:  
                total_pop_value_2013 = row.getValue('DP05_0001E')
                if total_pop_value_2013 != '0':
                    total_pop_list_2013.append(int(total_pop_value_2013))  

            total_pop_sum_2013 = sum( total_pop_list_2013)

            print(total_pop_sum_2013, " = total 2013 population in the subwatershed") 

            #find population change between 2018 and 2013
            population_change = total_pop_sum - total_pop_sum_2013

            population_change_list.append(population_change)
            print(population_change, " = population change between 2013 and 2018")




            #calculate drainage density
            #the total length of all streams and tributaries divided by basin area

            #all streams in subwatershed
            streams_clip = arcpy.Clip_analysis(streams, subwatershed, "streams")
            print("streams clipped")

            #add field for length
            streams_addfield = arcpy.AddField_management(streams_clip, "LENGTH", "DOUBLE")

            #calculate geometry 
            streams_calculate = arcpy.CalculateGeometryAttributes_management(streams_addfield, [["LENGTH", "LENGTH_GEODESIC"]], 
                                                                             "KILOMETERS")
            print("length calculated")

            #get the sum of all of the areas
            stream_length_list = []  

            rows = arcpy.SearchCursor(streams_calculate)  
            for row in rows:  
                stream_length = row.getValue("LENGTH")  
                stream_length_list.append(stream_length)  

            stream_length_sum = sum(stream_length_list)

            print(stream_length_sum, " = length of all streams in subwatershed")

            print(area, " = subwatershed area")

            drainage_density = stream_length_sum / area

            drainage_density_list.append(drainage_density)

            print(drainage_density, "is the drainage density (streams/km)")

            #find watershed length
            #watershed length = distance from outlet to watershed boundary along the main channel
            #we are assuming that the longest stream above is the main channel

            watershed_length = max(stream_length_list)

            watershed_length_list.append(watershed_length)

            print(watershed_length, " = watershed length (longest stream in subwatershed)")


            #calculate shape factor 
            #watershed length squared divided by watershed area

            shape_factor = (watershed_length**2) / area

            shape_factor_list.append(shape_factor)

            print(shape_factor, " is the shape factor.")



            #calculate relief ratio
            #relief divided by watershed length
            #length is is kilometers, convert to meters

            watershed_length_meters = watershed_length * 1000

            relief_ratio = relief / watershed_length_meters

            relief_ratio_list.append(relief_ratio)

            print(relief_ratio, " = relief ratio")


            #calculate ruggedness number
            #product of relief and drainage density

            #relief is in meters, convert first to km
            relief_km = relief / 1000
            print(relief_km, " is the relief in km")

            ruggedness = relief_km * drainage_density

            ruggedness_list.append(ruggedness)
            print(ruggedness, " is the ruggedness number")


            #calculate elongation ratio
            #ratio of diameter of a circle having the same area as the basin to the max basin length


            #diameter of circle with same area
            #A = (pi)r^2
            #r = sqrt(A/pi)
            #d = r*2

            radius = numpy.sqrt(area/numpy.pi)
            print(radius, " miles is the radius of a circle with the same area.")

            diameter = radius*2
            print(diameter, " miles is the diameter of a circle with the same area.")

            elongation_ratio = diameter / watershed_length

            elongation_ratio_list.append(elongation_ratio)


            #get number of buildings inside the aae zone

            buildings_aae_select = arcpy.SelectLayerByLocation_management(building_footprints_clip, "INTERSECT", nfhl_sfha_aae )

            buildings_aae = arcpy.CopyFeatures_management(buildings_aae_select, "buildings_aae.shp")

            buildings_aae_count = arcpy.GetCount_management(buildings_aae)

            buildings_aae_list.append(buildings_aae_count)

            #get number of buildings inside the x zone

            buildings_x_select = arcpy.SelectLayerByLocation_management(building_footprints_clip, "INTERSECT", nfhl_sfha_x )

            buildings_x = arcpy.CopyFeatures_management(buildings_x_select, "buildings_x.shp")

            buildings_x_count = arcpy.GetCount_management(buildings_x)

            buildings_x_list.append(buildings_x_count)

            #get area of various land use codes
            #clip usa land use polygon to the subwatershed
            lu_subwatershed = arcpy.Clip_analysis(lu_usa, subwatershed, "lu_subwatershed.shp")

            lu_subwatershed_area = arcpy.AddGeometryAttributes_management(lu_subwatershed, "AREA_GEODESIC", '',
                                                              "SQUARE_KILOMETERS")


            #lu 21 = developed open space
            lu_21_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                                 "gridcode = 21")

            lu_21 = arcpy.CopyFeatures_management(lu_21_select, "lu_21.shp")

            #get the sum of all of the areas
            lu_21_polygons_list = []  

            rows = arcpy.SearchCursor(lu_21)  
            for row in rows:  
                lu_21_polygon = row.getValue("AREA_GEO")  
                lu_21_polygons_list.append(lu_21_polygon)  

            lu_21_area_sum = sum(lu_21_polygons_list)

            lu_21_list.append(lu_21_area_sum)



            #lu 22 = developed low intensity
            lu_22_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                                 "gridcode = 22")

            lu_22 = arcpy.CopyFeatures_management(lu_22_select, "lu_22.shp")

            #get the sum of all of the areas
            lu_22_polygons_list = []  

            rows = arcpy.SearchCursor(lu_22)  
            for row in rows:  
                lu_22_polygon = row.getValue("AREA_GEO")  
                lu_22_polygons_list.append(lu_22_polygon)  

            lu_22_area_sum = sum(lu_22_polygons_list)

            lu_22_list.append(lu_22_area_sum)


            #lu 23 = developed medium intensity
            lu_23_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                                 "gridcode = 23")

            lu_23 = arcpy.CopyFeatures_management(lu_23_select, "lu_23.shp")

            #get the sum of all of the areas
            lu_23_polygons_list = []  

            rows = arcpy.SearchCursor(lu_23)  
            for row in rows:  
                lu_23_polygon = row.getValue("AREA_GEO")  
                lu_23_polygons_list.append(lu_23_polygon)  

            lu_23_area_sum = sum(lu_23_polygons_list)

            lu_23_list.append(lu_23_area_sum)



            #lu 24 = developed high intensity
            lu_24_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                                 "gridcode = 24")

            lu_24 = arcpy.CopyFeatures_management(lu_24_select, "lu_24.shp")

            #get the sum of all of the areas
            lu_24_polygons_list = []  

            rows = arcpy.SearchCursor(lu_24)  
            for row in rows:  
                lu_24_polygon = row.getValue("AREA_GEO")  
                lu_24_polygons_list.append(lu_24_polygon)  

            lu_24_area_sum = sum(lu_24_polygons_list)

            lu_24_list.append(lu_24_area_sum)



            #lu 41 = deciduous forest
            lu_41_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                                 "gridcode = 41")

            lu_41 = arcpy.CopyFeatures_management(lu_41_select, "lu_41.shp")

            #get the sum of all of the areas
            lu_41_polygons_list = []  

            rows = arcpy.SearchCursor(lu_41)  
            for row in rows:  
                lu_41_polygon = row.getValue("AREA_GEO")  
                lu_41_polygons_list.append(lu_41_polygon)  

            lu_41_area_sum = sum(lu_41_polygons_list)

            lu_41_list.append(lu_41_area_sum)


            #lu 82 = cultivated crops
            lu_82_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                                 "gridcode = 82")

            lu_82 = arcpy.CopyFeatures_management(lu_82_select, "lu_82.shp")

            #get the sum of all of the areas
            lu_82_polygons_list = []  

            rows = arcpy.SearchCursor(lu_82)  
            for row in rows:  
                lu_82_polygon = row.getValue("AREA_GEO")  
                lu_82_polygons_list.append(lu_82_polygon)  

            lu_82_area_sum = sum(lu_82_polygons_list)

            lu_82_list.append(lu_82_area_sum)



            #get percent impervious indicator for subwatershed area
            avg_impervious_pct = arcpy.sa.ZonalStatistics(subwatershed, "FID", impervious_usa, "MEAN" )

            #get avg value  value
            avg_impervious_result = arcpy.GetRasterProperties_management(avg_impervious_pct, "MAXIMUM")

            avg_impervious_pct_value = avg_impervious_result.getOutput(0)
            print(avg_impervious_pct_value, " = avg impervious percent ")

            impervious_percent_list.append(avg_impervious_pct_value)



            #get distance from residential buildings to streams

            building_footprints_residential_copy = arcpy.CopyFeatures_management(building_footprints_residential, "buildings_res.shp")


            #project buildings and streams so they are in the same GCS
            output_coord_system = arcpy.SpatialReference(r"C:\Users\jmatney\Documents\GitHub\IndianaRisk\data\Indiana_ML\working_dsn\NAD1983_ProjectionFile.prj")

            streams_project = arcpy.Project_management(streams_clip, "streams_project.shp",
                                                      output_coord_system)

            buildings_project = arcpy.Project_management(building_footprints_residential_copy, "buildings_res_project.shp",
                                                      output_coord_system)

            #use near tool to get distance to steams

            buildings_near = arcpy.Near_analysis(buildings_project, streams_project, "", "LOCATION", "", "GEODESIC")

            #get all the distances in to the streams
            dist_to_stream_list = []
            rows = arcpy.SearchCursor(buildings_near)  
            for row in rows:  
                dist_to_stream = row.getValue("NEAR_DIST")  
                dist_to_stream_list.append(dist_to_stream)  

            dist_to_stream_avg = mean(dist_to_stream_list)

            dist_to_stream_stdev = std(dist_to_stream_list)

            print("Distance to stream avg", dist_to_stream_avg)
            print("Distance to stream std", dist_to_stream_stdev)

            dist_to_stream_avg_list.append(dist_to_stream_avg)
            dist_to_stream_stdev_list.append(dist_to_stream_stdev)


         ## LOOP    

            #loop through all rainfall rasters in the same folder 


            arcpy.env.workspace = directory_rainfall

            rasters = arcpy.ListRasters("orb*")

            for raster in rasters:

                raster_name = os.path.basename(raster).rstrip(os.path.splitext(raster)[1])
                print("Raster name: ", raster_name)

                #get average rainfall for duration using zonal statistics
                #elevRaster = arcpy.sa.Raster(r"C:\Users\jmatney\Documents\GitHub\IndianaRisk\data\Indiana_ML\working_dsn\dem_clip.tif")

                centroid_rainfall = arcpy.sa.ExtractValuesToPoints(centroid, raster, r"C:\Users\jmatney\Documents\GitHub\IndianaRisk\data\shp\Centroid_Rainfall.shp", "NONE", "VALUE_ONLY")

                # Open a cursor on some fields in a table  
                rainfall_value = arcpy.da.SearchCursor(centroid_rainfall, ("RASTERVALU",)).next()[0]


        #         avg_rainfall = arcpy.sa.ZonalStatistics(subwatershed, "FID", raster, "MEAN" )

        #         #get avg value  value
        #         avg_result = arcpy.GetRasterProperties_management(avg_rainfall, "MAXIMUM")

                #rainfall_value = avg_result.getOutput(0)
                print(rainfall_value, " = avg rainfall in inches *1000")

                if raster_name == "orb100yr06h":
                    orb100yr06h_list.append(rainfall_value)

                elif raster_name == "orb100yr12h":
                    orb100yr12h_list.append(rainfall_value)

                elif raster_name == "orb100yr24h":
                    orb100yr24h_list.append(rainfall_value)

                elif raster_name == "orb25yr06h":
                    orb25yr06h_list.append(rainfall_value)

                elif raster_name == "orb25yr12h":
                    orb25yr12h_list.append(rainfall_value)

                elif raster_name == "orb25yr24h":
                    orb25yr24h_list.append(rainfall_value)

                elif raster_name == "orb2yr06h":
                    orb2yr06h_list.append(rainfall_value)

                elif raster_name == "orb2yr12h":
                    orb2yr12h_list.append(rainfall_value)

                elif raster_name == "orb2yr24h":
                    orb2yr24h_list.append(rainfall_value)

                elif raster_name == "orb50yr06h":
                    orb50yr06h_list.append(rainfall_value)

                elif raster_name == "orb50yr12h":
                    orb50yr12h_list.append(rainfall_value)

                elif raster_name == "orb50yr24h":
                    orb50yr24h_list.append(rainfall_value)

                elif raster_name == "orb100yr06ha_am":
                    orb100yr06ha_am_list.append(rainfall_value)

                elif raster_name == "orb100yr12ha_am":
                    orb100yr12ha_am_list.append(rainfall_value)

                elif raster_name == "orb100yr24ha_am":
                    orb100yr24ha_am_list.append(rainfall_value)

                elif raster_name == "orb25yr06ha_am":
                    orb25yr06ha_am_list.append(rainfall_value)

                elif raster_name == "orb25yr12ha_am":
                    orb25yr12ha_am_list.append(rainfall_value)

                elif raster_name == "orb25yr24ha_am":
                    orb25yr24ha_am_list.append(rainfall_value)

                elif raster_name == "orb2yr06ha_am":
                    orb2yr06ha_am_list.append(rainfall_value)

                elif raster_name == "orb2yr12ha_am":
                    orb2yr12ha_am_list.append(rainfall_value)

                elif raster_name == "orb2yr24ha_am":
                    orb2yr24ha_am_list.append(rainfall_value)

                elif raster_name == "orb50yr06ha_am":
                    orb50yr06ha_am_list.append(rainfall_value)

                elif raster_name == "orb50yr12ha_am":
                    orb50yr12ha_am_list.append(rainfall_value)

                elif raster_name == "orb50yr24ha_am":
                    orb50yr24ha_am_list.append(rainfall_value)

                else:
                    continue 



            #set workspace environment back to the newly created folder
            arcpy.env.workspace = output_workspace
            print(round(((time()-st)/60), 2) , 'minutes to process.')  


    
    
#########################################################################################


    
    
print(round(((time()-st)/60), 2) , 'minutes to process.')    
    

------ Subwatershed: 051201020406 Number 533 of 1590 ------
clipped dem to subwatershed area
31.8736279368  square kilometers
perimeter calculated
31.94915063  kilometers
2.7624123096466  = avg slope (%)
5.084865250352042 is the radius of a circle with the same perimeter.
81.22856290837503 is the area of a circle with the same perimeter.
0.39239433513988325  is the circulatory ratio
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
   FID  FREQUENCY  MAX_Z  MIN_Z
0    0       3505  255.0  204.0
255.0
204.0
The relief of the subwatershed is:  51.0  meters.
6.9150123517830995  = area of all AE and A zones in the subwatershed (square km)
0.01621263142287  = area of all X zones, 0.2PCT zone subtype in the subwatershed (square km)
3.0712655573410967  = area of all water bodies in the subwatershed (square km)
1  = number of dams in the subwatershed
1  = number of bridges in the subwatershed
34.65265840654073  = sum of all streets in the subwatershed (

1.5095203999531668  = area of all water bodies in the subwatershed (square km)
0  = number of dams in the subwatershed
0  = number of bridges in the subwatershed
4.223716047789  = sum of all streets in the subwatershed (km)
0  = sum of all railroads in the subwatershed (km)
6843  = total population in the subwatershed
39.1  = total percent of dependent population in the subwatershed
39.58504195806613  = population density of subwatershed (people/square km)
59847.0  = average median income in the subwatershed
6  = number of residential building footprints in the subwatershed
0.03470849799041309  = housing density (buildings per square km)
6993  = total 2013 population in the subwatershed
-150  = population change between 2013 and 2018
streams clipped
length calculated
35.35272701283851  = length of all streams in subwatershed
172.868327568  = subwatershed area
0.20450667574678796 is the drainage density (streams/km)
2.22528182556  = watershed length (longest stream in subwatershed)
0.02

streams clipped
length calculated
98.48934333185491  = length of all streams in subwatershed
58.8046298776  = subwatershed area
1.6748569549176213 is the drainage density (streams/km)
2.9441304613  = watershed length (longest stream in subwatershed)
0.1474017299521572  is the shape factor.
0.02988997979428637  = relief ratio
0.088  is the relief in km
0.14738741203275066  is the ruggedness number
4.326441383333281  miles is the radius of a circle with the same area.
8.652882766666561  miles is the diameter of a circle with the same area.
0.28814959526062  = avg impervious percent 
Distance to stream avg 213.7940347920062
Distance to stream std 145.58464804048887
Raster name:  orb100yr06h
5112  = avg rainfall in inches *1000
Raster name:  orb100yr06ha_am
5091  = avg rainfall in inches *1000
Raster name:  orb100yr12h
5749  = avg rainfall in inches *1000
Raster name:  orb100yr12ha_am
5726  = avg rainfall in inches *1000
Raster name:  orb100yr24h
6754  = avg rainfall in inches *1000
Raster

Raster name:  orb100yr06h
4419  = avg rainfall in inches *1000
Raster name:  orb100yr06ha_am
4401  = avg rainfall in inches *1000
Raster name:  orb100yr12h
5023  = avg rainfall in inches *1000
Raster name:  orb100yr12ha_am
5002  = avg rainfall in inches *1000
Raster name:  orb100yr24h
5690  = avg rainfall in inches *1000
Raster name:  orb100yr24ha_am
5667  = avg rainfall in inches *1000
Raster name:  orb25yr06h
3525  = avg rainfall in inches *1000
Raster name:  orb25yr06ha_am
3510  = avg rainfall in inches *1000
Raster name:  orb25yr12h
4067  = avg rainfall in inches *1000
Raster name:  orb25yr12ha_am
4050  = avg rainfall in inches *1000
Raster name:  orb25yr24h
4667  = avg rainfall in inches *1000
Raster name:  orb25yr24ha_am
4648  = avg rainfall in inches *1000
Raster name:  orb2yr06h
2087  = avg rainfall in inches *1000
Raster name:  orb2yr06ha_am
1921  = avg rainfall in inches *1000
Raster name:  orb2yr12h
2468  = avg rainfall in inches *1000
Raster name:  orb2yr12ha_am
2272  = avg

4960  = avg rainfall in inches *1000
Raster name:  orb25yr24ha_am
4940  = avg rainfall in inches *1000
Raster name:  orb2yr06h
2190  = avg rainfall in inches *1000
Raster name:  orb2yr06ha_am
2016  = avg rainfall in inches *1000
Raster name:  orb2yr12h
2543  = avg rainfall in inches *1000
Raster name:  orb2yr12ha_am
2341  = avg rainfall in inches *1000
Raster name:  orb2yr24h
2878  = avg rainfall in inches *1000
Raster name:  orb2yr24ha_am
2650  = avg rainfall in inches *1000
Raster name:  orb50yr06h
4345  = avg rainfall in inches *1000
Raster name:  orb50yr06ha_am
4327  = avg rainfall in inches *1000
Raster name:  orb50yr12h
4939  = avg rainfall in inches *1000
Raster name:  orb50yr12ha_am
4919  = avg rainfall in inches *1000
Raster name:  orb50yr24h
5614  = avg rainfall in inches *1000
Raster name:  orb50yr24ha_am
5591  = avg rainfall in inches *1000
5.8 minutes to process.
------ Subwatershed: 071200020710 Number 542 of 1590 ------
clipped dem to subwatershed area
79.7974576233  squ

...

5779  = total population in the subwatershed
43.55  = total percent of dependent population in the subwatershed
139.3961900837851  = population density of subwatershed (people/square km)
49970.5  = average median income in the subwatershed
157  = number of residential building footprints in the subwatershed
3.787022295060436  = housing density (buildings per square km)
5147  = total 2013 population in the subwatershed
632  = population change between 2013 and 2018
streams clipped
length calculated
32.035197587937006  = length of all streams in subwatershed
41.457374097  = subwatershed area
0.7727261623705971 is the drainage density (streams/km)
5.18755334075  = watershed length (longest stream in subwatershed)
0.649117563504193  is the shape factor.
0.006746918576251172  = relief ratio
0.035  is the relief in km
0.027045415682970902  is the ruggedness number
3.632670096539858  miles is the radius of a circle with the same area.
7.265340193079716  miles is the diameter of a circle with 

0.88512307405472  = avg impervious percent 
Distance to stream avg 417.41476739215983
Distance to stream std 464.06186371765097
Raster name:  orb100yr06h
4160  = avg rainfall in inches *1000
Raster name:  orb100yr06ha_am
4143  = avg rainfall in inches *1000
Raster name:  orb100yr12h
4799  = avg rainfall in inches *1000
Raster name:  orb100yr12ha_am
4779  = avg rainfall in inches *1000
Raster name:  orb100yr24h
5627  = avg rainfall in inches *1000
Raster name:  orb100yr24ha_am
5604  = avg rainfall in inches *1000
Raster name:  orb25yr06h
3396  = avg rainfall in inches *1000
Raster name:  orb25yr06ha_am
3382  = avg rainfall in inches *1000
Raster name:  orb25yr12h
3954  = avg rainfall in inches *1000
Raster name:  orb25yr12ha_am
3938  = avg rainfall in inches *1000
Raster name:  orb25yr24h
4610  = avg rainfall in inches *1000
Raster name:  orb25yr24ha_am
4591  = avg rainfall in inches *1000
Raster name:  orb2yr06h
2079  = avg rainfall in inches *1000
Raster name:  orb2yr06ha_am
1914  = a

4587  = avg rainfall in inches *1000
Raster name:  orb25yr12ha_am
4568  = avg rainfall in inches *1000
Raster name:  orb25yr24h
5383  = avg rainfall in inches *1000
Raster name:  orb25yr24ha_am
5361  = avg rainfall in inches *1000
Raster name:  orb2yr06h
2243  = avg rainfall in inches *1000
Raster name:  orb2yr06ha_am
2065  = avg rainfall in inches *1000
Raster name:  orb2yr12h
2643  = avg rainfall in inches *1000
Raster name:  orb2yr12ha_am
2433  = avg rainfall in inches *1000
Raster name:  orb2yr24h
3093  = avg rainfall in inches *1000
Raster name:  orb2yr24ha_am
2848  = avg rainfall in inches *1000
Raster name:  orb50yr06h
4573  = avg rainfall in inches *1000
Raster name:  orb50yr06ha_am
4554  = avg rainfall in inches *1000
Raster name:  orb50yr12h
5232  = avg rainfall in inches *1000
Raster name:  orb50yr12ha_am
5211  = avg rainfall in inches *1000
Raster name:  orb50yr24h
6144  = avg rainfall in inches *1000
Raster name:  orb50yr24ha_am
6119  = avg rainfall in inches *1000
231.17 

4447  = avg rainfall in inches *1000
Raster name:  orb50yr12h
5084  = avg rainfall in inches *1000
Raster name:  orb50yr12ha_am
5063  = avg rainfall in inches *1000
Raster name:  orb50yr24h
5903  = avg rainfall in inches *1000
Raster name:  orb50yr24ha_am
5879  = avg rainfall in inches *1000
240.81 minutes to process.
------ Subwatershed: 051402020401 Number 647 of 1590 ------
clipped dem to subwatershed area
159.525396764  square kilometers
perimeter calculated
87.0184180823  kilometers
1.7731782197952  = avg slope (%)
13.849411377835214 is the radius of a circle with the same perimeter.
602.5769347351136 is the area of a circle with the same perimeter.
0.26473863762164357  is the circulatory ratio
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
   FID  FREQUENCY  MAX_Z  MIN_Z
0    0       8905  173.0  105.0
173.0
105.0
The relief of the subwatershed is:  68.0  meters.
34.569858261679215  = area of all AE and A zones in the subwatershed (squa

KeyboardInterrupt: 

In [14]:
st = time()
outputs = {'subwatershed': subwatershed_list[0:114],
          'area': area_list[0:114],
          'perimeter': perimeter_list[0:114],
          'circulatory_ratio': circulatory_ratio_list[0:114],
          'relief': relief_list[0:114],
          'avg_slope': avg_slope_list[0:114],
           'watershed_length': watershed_length_list[0:114],
           'elongation_ratio': elongation_ratio_list[0:114],
           'drainage_density': drainage_density_list[0:114],
           'shape_factor': shape_factor_list[0:114],
           'relief_ratio': relief_ratio_list[0:114],
           'ruggedness': ruggedness_list[0:114],
           'aae_area': aae_list[0:114],
           'buildings_aae_count': buildings_aae_list[0:114],
           'x_area': x_list[0:114],
           'buildings_x_count': buildings_x_list[0:114],
           'water_bodies_area': water_bodies_list[0:114],
           'dams_count': dams_list[0:114],
           'bridges_count': bridges_list[0:114],
           'streets_km': streets_list[0:114],
           'railroads_km': railroads_list[0:114],
           'population': population_list[0:114],
           'population_density': population_density_list[0:114],
           'avg_median_income': avg_median_income_list[0:114],
           'housing_density': housing_density_list[0:114],
           'population_change': population_change_list[0:114],
           'dependent_population_pct': dependent_population_list[0:114],
           'dist_to_stream_avg (m)': dist_to_stream_avg_list[0:114],
           'dist_to_stream_stdev (m)': dist_to_stream_stdev_list[0:114],
           'lu_21_area' : lu_21_list[0:114],
           'lu_22_area' : lu_22_list[0:114],
           'lu_23_area' : lu_23_list[0:114],
           'lu_24_area': lu_24_list[0:114],
           'lu_41_area': lu_41_list[0:114],
           'lu_82_area': lu_82_list[0:114],
           'avg_impervious_percent': impervious_percent_list[0:114],
           'orb100yr06h': orb100yr06h_list[0:114],
           'orb100yr12h': orb100yr12h_list[0:114],
           'orb100yr24h': orb100yr24h_list[0:114],
           'orb25yr06h': orb25yr06h_list[0:114],
           'orb25yr12h': orb25yr12h_list[0:114],
           'orb25yr24h':orb25yr24h_list[0:114],
           'orb2yr06h': orb2yr06h_list[0:114],
           'orb2yr12h': orb2yr12h_list[0:114],
           'orb2yr24h': orb2yr24h_list[0:114],
           'orb50yr06h': orb50yr06h_list[0:114],
           'orb50yr12h': orb50yr12h_list[0:114],
           'orb50yr24h':orb50yr24h_list[0:114],
           'orb100yr06ha_am': orb100yr06ha_am_list[0:114],
           'orb100yr12ha_am': orb100yr12ha_am_list[0:114],
           'orb100yr24ha_am': orb100yr24ha_am_list[0:114],
           'orb25yr06ha_am': orb25yr06ha_am_list[0:114],
           'orb25yr12ha_am': orb25yr12ha_am_list[0:114],
           'orb25yr24ha_am': orb25yr24ha_am_list[0:114],
           'orb2yr06ha_am': orb2yr06ha_am_list[0:114],
           'orb2yr12ha_am': orb2yr12ha_am_list[0:114],
           'orb2yr24ha_am': orb2yr24ha_am_list[0:114],
           'orb50yr06ha_am': orb50yr06ha_am_list[0:114],
           'orb50yr12ha_am': orb50yr12ha_am_list[0:114],
           'orb50yr24ha_am': orb50yr24ha_am_list[0:114]
          }

# 
# 
# 
#  
#         
# 
            
outputs_df = pd.DataFrame(outputs, columns = ['subwatershed',
                                             'area',
                                             'perimeter',
                                             'circulatory_ratio',
                                             'relief',
                                             'avg_slope',
                                              'watershed_length',
                                              'elongation_ratio',
                                              'drainage_density',
                                              'shape_factor',
                                              'relief_ratio',
                                              'ruggedness',
                                              'aae_area',
                                              'buildings_aae_count',
                                              'x_area',
                                              'buildings_x_count',
                                              'water_bodies_area',
                                              'dams_count',
                                              'bridges_count',
                                              'streets_km',
                                              'railroads_km',
                                              'population',
                                              'population_density',
                                              'avg_median_income',
                                              'housing_density',
                                              'population_change',
                                              'dependent_population_pct',
                                              'dist_to_stream_avg (m)',
                                              'dist_to_stream_stdev (m)',
                                              'lu_114_area',
                                              'lu_22_area',
                                              'lu_23_area',
                                              'lu_24_area',
                                              'lu_41_area',
                                              'lu_82_area',
                                              'avg_impervious_percent',
                                             'orb100yr06h',
                                              'orb100yr12h',
                                              'orb100yr24h',
                                              'orb25yr06h',
                                              'orb25yr12h',
                                              'orb25yr24h',
                                              'orb2yr06h',
                                              'orb2yr12h',
                                              'orb2yr24h',
                                              'orb50yr06h',
                                              'orb50yr12h',
                                              'orb50yr24h',
                                              'orb100yr06ha_am',
                                              'orb100yr12ha_am',
                                              'orb100yr24ha_am',
                                              'orb25yr06ha_am',
                                              'orb25yr12ha_am',
                                              'orb25yr24ha_am',
                                              'orb2yr06ha_am',
                                              'orb2yr12ha_am',
                                              'orb2yr24ha_am',
                                              'orb50yr06ha_am',
                                              'orb50yr12ha_am',
                                              'orb50yr24ha_am'
                                             ])    
    

#     
#  
# 
# 
# 
#




print(outputs_df)

outputs_df.to_excel(r"C:\Users\jmatney\Documents\GitHub\IndianaRisk\data\output\IN_subwatershed_data___.xlsx")
print(round(((time()-st)/60), 2) , 'minutes to process.')

     subwatershed        area  perimeter  circulatory_ratio  relief  \
0    051201020406   31.873628  31.949151           0.392394    51.0   
1    040500011602   49.797378  51.032132           0.240286     0.0   
2    051401041001  172.868328  84.839591           0.301806   135.0   
3    040400010102   42.069922  47.198018           0.237320    39.0   
4    051202080401   58.804630  42.324744           0.412508    88.0   
..            ...         ...        ...                ...     ...   
109  050800020802   47.456439  37.344570           0.427612     0.0   
110  051202080805   62.478389  40.986326           0.467371   107.0   
111  051201081606   66.584437  42.751504           0.457804    60.0   
112  051201100402   70.340447  57.610779           0.266322    12.0   
113  051201061006   49.556842  40.312159           0.383214    21.0   

             avg_slope  watershed_length  elongation_ratio  drainage_density  \
0      2.7624123096466          3.004907          2.120019         

In [13]:
len(orb50yr24ha_am_list) 

114